In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import re
import json
import sys
import datetime 
import matplotlib.pyplot as plt
import math
import statistics
from pylab import rcParams
from scipy.signal import lfilter, filtfilt
from collections import defaultdict
from collections import Counter
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import string
import time

rf = pickle.load(open('activityID_rf.sav', 'rb'))

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
RANKING_diabetes = {
    '003':20,
    '009':21,
    '007':22,
    '005':23,
    '006':24,
    '008':26,
    '004':27,
    '001':28,
    '002':29,
    
}
RANKING_healthy = {
    '016':1,
    '002':2,
    '005':3,
    '008':4,
    '003':5,
    '019':6,
    '009':7,
    '010':8,
    '020':9,
    '006':10,
    '018':11,
    '013':12,
    '015':13,
    '007':14,
    '014':15,
    '017':16,
    '004':17,
    '011':18,
    '001':19,
    '012':25,
}
RANKING_healthy_num = {
    16:1,
    2:2,
    5:3,
    8:4,
    3:5,
    19:6,
    9:7,
    10:8,
    20:9,
    6:10,
    18:11,
    13:12,
    15:13,
    7:14,
    14:15,
    17:16,
    4:17,
    11:18,
    1:19,
    12:25,
}
RANKING_COMPOSE = {
    10:1,
    13:2,
    19:3,
    15:4,
    18:5,
    5:6,
    17:7,
    2:8,
    9:9,
    12:10,
    16:11,
    20:12,
    7:13,
    8:14,
    11:15,
    1:16,
    14:17,
    6:18,
    3:19,
    4:20
    
}

RANKING_COMPOSE_DOCS = {
    5:1,
    10:2,
    19:3,
    15:4,
    9:5,
    18:6,
    8:7,
    13:8,
    17:9,
    2:10,
    1:11,
    12:12,
    16:13,
    20:14,
    7:15,
    6:16,
    14:17,
    11:18,
    3:19,
    4:20
}


WEIGHTED_RANKING_healthy = {
    16:1,
    2:2,
    5:3,
    8:4,
    3:5,
    19:20,
    9:21,
    10:22,
    20:23,
    6:23,
    18:25,
    13:26,
    15:27,
    7:28,
    14:29,
    17:30,
    4:31,
    11:32,
    1:33,
    12:50,
}


In [ ]:
for i in overall_data.groupby('ranking', sort=):
    print(i[0])

# Helper functions to load the d1namo dataset, classify activity and extract features


In [3]:
#rf model
#rf = pickle.load(open('activityID_rf.sav', 'rb'))

#constants
WINDOWSIZE_MINUTES  = 10
SEDENTARY_THRESHOLD = 0.02
ACTIVE_THRESHOLD    = 0.65
SAMPLE_SECONDS      = 6 
SAMPLING_RATE       = 100 #Hz
SAMPLE_OVERLAP      = 0.5

#lists
COLUMNS = [
    'Time',
    'subject_id',
    
    #activity domain features
    'breathing_rate',

    's_avg_activity',
    's_activityLvl',
    

    'c_main_activity',
    'c_activityLvl',
    
    #heart health domain features
    'heart_rate_max_min',
    'heart_rate_max_min_day',
    'standard_deviation_HR',
    'standard_deviation_HR_day',
    'standard_deviation_of_the_interbeat_interval',
    'average_resting_heart_rate',
    'average_resting_heart_rate_day',
    'average_heart_rate_by_s_activity',
    'proportion_low_activity_above_resting_HR',
    'average_heart_rate_day',
    
    #diet domain features
    'glucose_in_window',
    'calories_per_day',
    'unbalanced_diet_window',
    'bad_quality_diet_window',   
]

CORE_FEATURES = [
        "x_vert_accel_16g_chest",
        "y_lat_accel_16g_chest",
        "z_sagit_accel_16g_chest",
    ]

activity_lvls = {
        0 : 1,
        1 : 0,
        2 : 0,
        3 : 1,
        4 : 1,
        5 : 2,
        6 : 2,
        7 : 2,
        9 : 0,
        10: 0,
        11: 0,
        12: 2,
        13: 1,
        16: 1,
        17: 1,
        18: 1,
        19: 1,
        20: 2,
        24: 2,


}


#Funtions
def extract_features(accel_file_path,summary_file_path, subject_no, diet_params_arr):
    summary_file   = pd.read_csv(summary_file_path)
    current_day    = pd.to_datetime(summary_file.Time[1][1:10], dayfirst=True).date().strftime("%Y-%m-%d")
    start_time     = pd.to_datetime(summary_file.Time[1]).time().strftime("%H:%M")
    end_time       = pd.to_datetime(summary_file.Time[len(summary_file)-1]).time().strftime("%H:%M")
    accel_features = generate_acceleration_features(accel_file_path)
    
    summary_file['c_activity_main'] = pd.Series(np.repeat(rf.predict(accel_features), 3))
    summary_file['c_activity_main'] = summary_file['c_activity_main'].fillna(0)
    summary_file['c_ActivityLvl']   = [activity_lvls[x] for x in summary_file.c_activity_main]
    
    day_features_temp = pd.DataFrame()
    day_features_temp = day_features_temp.append(extract_activity_domain_features(summary_file),\
                                                 ignore_index = True)
    day_features_temp = pd.concat([day_features_temp, \
                                   extract_heart_domain_features(summary_file)], axis=1) 
    day_features_temp = pd.concat([day_features_temp, \
                                   extract_diet_domain_features(\
                                                                diet_params_arr
                                                                , current_day
                                                                , start_time
                                                                , end_time)], axis=1)
    
    day_features_temp['subject_id'] = subject_no

    return day_features_temp


def generate_acceleration_features(accel_file_path):
    accel_df = pd.read_csv(accel_file_path)
    accel_df = accel_df[accel_df.notna()] #remove NAs 
    """
    Transforming the data by subtracting 2048 from the sensor readings, dividing by 83 
    to get the acceleration in m/s² and then multiplying by the gravity constant to get the acceleration 
    in g, as in the dataset used for training & testing the activity classifier. To keep the timestamps, the process 
    comprises two steps
    """
    transformed_accel_df =(((accel_df.iloc[:,1:4]-2048)/83)*9,81)[0]
    transformed_accel_df = (transformed_accel_df
                              .rename(columns = {"Vertical": CORE_FEATURES[0],
                                                 "Lateral" : CORE_FEATURES[1],
                                                 "Sagittal": CORE_FEATURES[2]
                                                }))
    return create_features_from_dataset(transformed_accel_df)
    

    
    
def extract_activity_domain_features( summary_file):
    summary_file['s_activityLvl'] = calculate_summary_activity_based_activity_level(summary_file.Activity)
    activity_domain_features      = pd.DataFrame()
    
    window_summary = {}
    samples        = window_df(summary_file,WINDOWSIZE_MINUTES*60 , overlap=0)

    for idx,sample in enumerate(samples):

        counted_c_activities      = Counter(sample.c_ActivityLvl)
        counted_s_activities      = Counter(sample.s_activityLvl)
        valid_activity            = sample[sample.Activity>0].Activity.mean()
        valid_BR                  = sample[(sample.BR>4)&(sample.BR<70)].BR.mean()
        window_summary['Time']    = sample.Time.iloc[0]
        window_summary['breathing_rate' ]   = valid_BR
        window_summary['s_avg_activity' ]   = valid_activity
        window_summary['c_activityLvl'  ]   = Counter(sample.c_ActivityLvl).most_common(1)[0][0]
        window_summary['c_main_activity']   = (counted_c_activities.most_common(1)[0][0]\
                    if (counted_c_activities.most_common(1)[0][0] !=0)|(len(counted_c_activities)==1) \
                    else counted_c_activities.most_common(2)[1][0]) 
        window_summary['s_activityLvl'  ]   = counted_s_activities.most_common(1)[0][0]
        window_summary['s_breathing_by_act']= valid_BR / valid_activity

        activity_domain_features  = activity_domain_features.append(window_summary, ignore_index=True)
        
    return activity_domain_features
    
    

    

def extract_heart_domain_features(summary_file): 
    heart_domain_features  = pd.DataFrame()
    valid_summary_file     = summary_file[(summary_file.HR>25) & (summary_file.HR<240)]
    
    window_summary         = {}
    samples                = window_df(summary_file,WINDOWSIZE_MINUTES*60 , overlap=0)
    
    heart_rate_max_min_day = valid_summary_file.HR.max() - valid_summary_file.HR.min()
    heart_rate_std_day     = valid_summary_file.HR.std()
    hrv_variability        = calculate_standard_deviation_of_the_interbeat_interval(valid_summary_file.HRV)
    average_resting_heart_rate = valid_summary_file[valid_summary_file.Activity<SEDENTARY_THRESHOLD].HR.mean()
    average_heart_rate_day = valid_summary_file.HR.mean()
    
    for idx,sample in enumerate(samples):
            sample = sample[(sample.HR>25) & (sample.HR<240)]
            if len(sample)>0:
                window_summary['heart_rate_max_min']         = sample.HR.max() - sample.HR.min()
                window_summary['heart_rate_max_min_day']     = heart_rate_max_min_day
                window_summary['sandard_deviation_HR_day']   = heart_rate_std_day
                window_summary['standard_deviation_HR']       = sample.HR.std()
                window_summary['standard_deviation_of_the_interbeat_interval'] = hrv_variability
                window_summary['average_resting_heart_rate']       =sample[sample.Activity<SEDENTARY_THRESHOLD].HR.mean()
                window_summary['average_resting_heart_rate_day']   = average_resting_heart_rate
                window_summary['average_heart_rate_day']     = average_heart_rate_day
                window_summary['proportion_low_activity_above_resting_HR']= len(sample[\
                                                                                       (sample.Activity <= SEDENTARY_THRESHOLD)& \
                                                                                       (sample.HR >= average_resting_heart_rate)
                                                                                      ])\
                                                                            /len(sample)
            else:
                window_summary['heart_rate_max_min']         = -1
                window_summary['heart_rate_max_min_day']     = -1
                window_summary['sandard_deviation_HR_day']   = -1
                window_summary['standard_deviation_HR']       = -1
                window_summary['standard_deviation_of_the_interbeat_interval'] = -1
                window_summary['average_resting_heart_rate']       = -1
                window_summary['average_resting_heart_rate_day']   = -1
                window_summary['average_heart_rate_day']     = -1
                window_summary['proportion_low_activity_above_resting_HR'] = -1
                
            heart_domain_features = heart_domain_features.append(window_summary, ignore_index = True) 
    return heart_domain_features


def calculate_standard_deviation_of_the_interbeat_interval(summary_file_HRV):
    samples          = window_df(summary_file_HRV,5*60 , overlap=0)
    window_summary   = np.array([])
    for idx,sample in enumerate(samples):
        sample = sample[sample<65535]
        window_summary = np.append(window_summary,sample.mean() )
        
    return window_summary[~np.isnan(window_summary)].mean()


def calculate_summary_activity_based_activity_level(summary_file_Activity):
    return_array = []
    for i in range(len(summary_file_Activity)):
        if summary_file_Activity[i]   <= SEDENTARY_THRESHOLD:
            return_array.append(0)
        elif summary_file_Activity[i] >= ACTIVE_THRESHOLD:
             return_array.append(2)
        else:
             return_array.append(1)
    return return_array
                                                                     
    
def extract_diet_domain_features(subject_diet_params,date,start_time,end_time ):
    print(f'gluco date: {date}')

    glucose, food      = subject_diet_params

    glucose_day        = glucose[glucose.date == date].reset_index()
    #food_day           = food[(food.date      == date) & (food.calories>0)].reset_index()
    starttime          = start_time
    endtime = end_time

    diary_df           = pd.DataFrame()

    n_10_min_windows   = round(((timedelta(hours = (time.strptime(endtime,'%H:%M'))\
                                                 .tm_hour, minutes = time.strptime(endtime,'%H:%M')\
                                                 .tm_min).total_seconds()/60) - \
                               (timedelta(hours = time.strptime(starttime,'%H:%M')\
                                                 .tm_hour, minutes = time.strptime(starttime,'%H:%M')\
                                                 .tm_min).total_seconds()/60))/WINDOWSIZE_MINUTES
                            )
    if n_10_min_windows < 0:
        overhead = (timedelta(hours = (time.strptime(endtime,'%H:%M'))\
                                                     .tm_hour, minutes = time.strptime(endtime,'%H:%M')\
                                                     .tm_min).total_seconds()/60)
        endtime = '23:59'
        min_diff = ((timedelta(hours = (time.strptime(endtime,'%H:%M'))\
                                                 .tm_hour, minutes = time.strptime(endtime,'%H:%M')\
                                                 .tm_min).total_seconds()/60) - \
                               (timedelta(hours = time.strptime(starttime,'%H:%M')\
                                                 .tm_hour, minutes = time.strptime(starttime,'%H:%M')\
                                                 .tm_min).total_seconds()/60))
        n_10_min_windows = round((min_diff+overhead)/WINDOWSIZE_MINUTES)




    glucose_day_time_array = np.append(\
                                        np.array([s.translate(str.maketrans('', '', string.punctuation)) \
                                                  for s in glucose_day.time]).astype(int)\
                                        ,99999)
    glucose_time_index  = 0

    #food_day_time_array = np.append(\
                                        #np.array([s.translate(str.maketrans('', '', string.punctuation))\
                                         #         for s in food_day.time]).astype(int)\
                                        #,99999)
    #food_time_index     = 0
    time_index          = int(starttime.translate(str.maketrans('', '', string.punctuation)))
    for i in range(1, n_10_min_windows):
        PGC_score       = 0 
        g1, g2          = glucose_day_time_array[glucose_time_index], glucose_day_time_array[glucose_time_index+1]
        #f1,f2           = food_day_time_array[food_time_index],food_day_time_array[food_time_index+1]
        if abs(g1 - time_index) > abs(g2 - time_index):
            glucose_time_index += 1 
        #if abs(f1 - time_index) > abs(f2 - time_index):
         #   food_time_index    += 1 
        
        #calculating a blood plasma glucose concentration score that exponentially punishes abnormal values 
        #in accordance with the NICE reccomendataions:
        #https://www.nice.org.uk/guidance/ng17/ifp/chapter/Testing-your-own-blood-glucose-and-target-levels
        if glucose_day.type[glucose_time_index]=='BB':
            PGC_score = ((abs(6 - glucose_day.glucose[glucose_time_index])-1)*10)**2 \
            if 6 - glucose_day.glucose[glucose_time_index]>1 \
            else 0
        elif glucose_day.type[glucose_time_index].startswith('B'):
            PGC_score = ((abs(6 - glucose_day.glucose[glucose_time_index])-1)*10)**2 \
            if 6.5 - glucose_day.glucose[glucose_time_index]>1.5 \
            else 0
        else:
            PGC_score = ((abs(6 - glucose_day.glucose[glucose_time_index])-1)*10)**2 \
            if 7 - glucose_day.glucose[glucose_time_index]>2 \
            else 0
            
        
        diary_df        = diary_df.append({
            'glucose'                : glucose_day.glucose[glucose_time_index],
            #'cals'                   : food_day.calories[food_time_index],
            #'unbalanced_diet_window' : food_day.calories[food_time_index] if food_day.balance[food_time_index]=='Unbalance' else 0,
            #'bad_quality_diet_window': food_day.calories[food_time_index] if food_day.quality[food_time_index]=='Low quality' else 0,
            'PGC_score'              : PGC_score
        }, ignore_index = True)
        time_index += 10
        if str(time_index)[-2] == '6':
            time_index += 40

    print(len(diary_df))
    return diary_df





def create_feature_table(path):
    os_path            = os.path.join(os.getcwd() + path)                 
    extracted_features = pd.DataFrame()
    all_subjects       = np.array([name for name in os.listdir(os_path)])

    for subject_no in all_subjects:

        print('At subject no ', subject_no,' time ',datetime.now().time().strftime("%H:%M:%S") )
        
        subject_path          = os.path.join(os_path + '/' + subject_no)
        day_paths_of_subject  = [os.path.join(subject_path + '/sensor_data/' + folder + '/' + folder) \
                                for folder \
                                in os.listdir(os.path.join(subject_path+'/sensor_data'))]
        glucose_record        = pd.read_csv(os.path.join(subject_path  + '/glucose.csv'))
        food_record           = pd.read_csv(os.path.join(subject_path  + '/food.csv'))
        subject_record        = pd.DataFrame()
        for day_path in day_paths_of_subject:
            
            accel_file_path   = str(day_path) + '_Accel.csv'
            summary_file_path = str(day_path) + '_Summary.csv'
            
            extracted_features_temp = extract_features(accel_file_path,\
                                 summary_file_path,\
                                 subject_no,\
                                 [glucose_record,food_record])
            
            extracted_features = extracted_features.append(extracted_features_temp, ignore_index = True)
            subject_record     = subject_record.append(extracted_features_temp, ignore_index = True)
            
        subject_record.to_csv(f'summaries_folder/summary_subject_{subject_no}_d.csv')   
    #extracted_features['ranking'] = [RANKING_healthy_num[i] for i in extracted_features.subject_id]
    return extracted_features

"""added the heart rate std to the core features"""
def create_features_from_dataset(df):
  
    # 0-center the mean and normalize. this adds about 5 points
    df[CORE_FEATURES] = standardize(df[CORE_FEATURES])
    
    new_group         = defaultdict(list)
    
    #creating a generator object
    samples           = window_df(df, SAMPLE_SECONDS*SAMPLING_RATE, SAMPLE_OVERLAP)


    for idx,sample in enumerate(samples):

        # means of amplitudes
        means = sample[CORE_FEATURES].mean()
        new_group['x_accel_mean'].append(means["x_vert_accel_16g_chest"])
        new_group['y_accel_mean'].append(means['y_lat_accel_16g_chest'])
        new_group['z_accel_mean'].append(means['z_sagit_accel_16g_chest'])

        # standard deviation of amplitudes
        stds = sample[CORE_FEATURES].std()
        new_group['x_accel_std'].append(stds["x_vert_accel_16g_chest"])
        new_group['y_accel_std'].append(stds['y_lat_accel_16g_chest'])
        new_group['z_accel_std'].append(stds['z_sagit_accel_16g_chest'])
        #new_group['heart_std'].append(sample.heart_rate.std()) #added a parameter for heart rate

        # max-min
        new_group['x_max_min'].append(max(sample.x_vert_accel_16g_chest) - min(sample.x_vert_accel_16g_chest))
        new_group['y_max_min'].append(max(sample.y_lat_accel_16g_chest) - min(sample.y_lat_accel_16g_chest))
        new_group['z_max_min'].append(max(sample.z_sagit_accel_16g_chest) - min(sample.z_sagit_accel_16g_chest))
        #new_group['heart_max_min'].append(max(sample.heart_rate) - min(sample.heart_rate) if
          #                                max(sample.heart_rate) - min(sample.heart_rate)>=0 else 0)

        # xy, xz, yz correlations
        corrs = sample[CORE_FEATURES].corr()
        new_group['xy_corr'].append(corrs.loc['x_vert_accel_16g_chest', 'y_lat_accel_16g_chest'])
        new_group['xz_corr'].append(corrs.loc['x_vert_accel_16g_chest', 'z_sagit_accel_16g_chest'])
        new_group['yz_corr'].append(corrs.loc['y_lat_accel_16g_chest', 'z_sagit_accel_16g_chest'])

        # root-mean-square(x, y, z)
        rms   = np.sqrt(np.mean(np.square(sample[CORE_FEATURES]), axis=1))
        new_group['rms_mean'].append(rms.mean())
        new_group['rms_std'].append(rms.std())


        # fourier transforms! 
        x_fft = abs_rfft(sample['x_vert_accel_16g_chest'])
        y_fft = abs_rfft(sample['y_lat_accel_16g_chest'])
        z_fft = abs_rfft(sample['z_sagit_accel_16g_chest'])

        new_group['x_fft_max'].append(x_fft.max())
        new_group['y_fft_max'].append(y_fft.max())
        new_group['z_fft_max'].append(z_fft.max())

        new_group['x_fft_min'].append(x_fft.min())
        new_group['y_fft_min'].append(y_fft.min())
        new_group['z_fft_min'].append(z_fft.min())

        new_group['x_fft_mean'].append(x_fft.mean())
        new_group['y_fft_mean'].append(y_fft.mean())
        new_group['z_fft_mean'].append(z_fft.mean())

        new_group['x_fft_std'].append(x_fft.std())
        new_group['y_fft_std'].append(y_fft.std())
        new_group['z_fft_std'].append(z_fft.std())
    
    new_group = pd.DataFrame(new_group)

    return new_group

def abs_rfft( series ):
    complex_fourier = np.fft.rfft( series )
    return np.absolute(complex_fourier)

def standardize(df):
    """
    Make the mean of data 0 with standard dev of 1
    """
    return (df - df.mean()) / df.std() 


# Helper functions for windowing the data
def window(width, overlap, max_idx):
    """
    Generates tuples of indices that define a window
    of given width and overlap. 
    
    For example:
    window(width=10, overlap=0.5, max_length=30)
    (0, 10)
    (5, 15)
    (10, 20)
    (15, 25)
    Note: it trims the end; i.e. won't return (25, 30)
    """
    start   = 0
    if overlap < 0.0 or overlap >= 1.:
        raise ValueError("overlap needs to be a number between 0 and 1")
    while True:
        end = start + width
        if end >= max_idx:
            return None
        yield start, end
        start  += max(int((1-overlap)*width), 1)

        
def window_df(df, width, overlap):
    """
    Applies window to a dataframe
    """
    windows = window(width, overlap, len(df))
    for start, end in windows:
        yield df[start:end]



In [65]:
#data cleanining
#pd.read_csv('healthy_subset/004/food.csv').drop([1,4,11]).reset_index(drop=True).to_csv( 'healthy_subset/004/food.csv')
#pd.read_csv('healthy_subset/002/food.csv').drop([25,27]).reset_index(drop=True).to_csv( 'healthy_subset/002/food.csv')
#pd.read_csv('healthy_subset/007/food.csv').drop([11,15]).reset_index(drop=True).to_csv( 'healthy_subset/007/food.csv')
#pd.read_csv('healthy_subset/013/food.csv').drop([5,8,10,11]).reset_index(drop=True).to_csv( 'healthy_subset/013/food.csv')
#pd.read_csv('healthy_subset/013/food.csv').drop([9]).reset_index(drop=True).to_csv( 'healthy_subset/013/food.csv')
#x = pd.read_csv('healthy_subset/017/glucose.csv')
#x.iloc[17,2]=7.0
#x.iloc[17,2]
#x.to_csv('healthy_subset/017/glucose.csv')
#x = pd.read_csv('healthy_subset/014/glucose.csv')
#x.iloc[0,3] ='BB' # assumed this value to be bb because the first meal was at 12:20, after another 'BL' measurement
#x.iloc[0,3]
#x.to_csv('healthy_subset/014/glucose.csv')
#x = pd.read_csv('healthy_subset/006/glucose.csv')
#pd.read_csv('healthy_subset/006/food.csv')
#x.iloc[0,3] ='BB' # assumed this value to be bb because the first meal was at 9:19
#x.iloc[1,3] = 'AB' 
#x.to_csv('healthy_subset/006/glucose.csv')
np.array([name for name in os.listdir('healthy_subset')])

array(['016', '017', '018', '009', '010', '004', '002', '015', '014',
       '020', '007', '019', '008', '003', '005', '006', '013', '011',
       '001'], dtype='<U3')

In [4]:
#Runing this takes more than two hours. Intermediate summaries per individual are therefore separatly, in case the
#algorythm stops, e.g. due to a an error or a n.a. value. 
x = create_feature_table('/healthy_subset')


At subject no  009  time  21:50:04


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:208: RuntimeWarning: Mean of empty slice.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


gluco date: 2014-10-02
85


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:208: RuntimeWarning: Mean of empty slice.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


gluco date: 2014-10-04
34


...

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


gluco date: 2014-10-02
102
At subject no  003  time  22:23:43


/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


gluco date: 2014-10-03


IndexError: index 1 is out of bounds for axis 0 with size 1

In [49]:
##the first part creates an overall summary  from the the individuals summary files in case the prior algorithm 
#was interrupted
x.to_csv('overall_summaryV2.0.csv')
x = pd.DataFrame()
for summary in os.listdir('summaries_folder'):
    x = x.append(pd.read_csv('summaries_folder/' + summary))

##finally the ranking from above is added as an additional column and rows with invalid values are removed:
x['ranking'] = [RANKING_healthy_num[i] for i in  x.subject_id]
prior_len = len(x)
x = x[~(x.cals==0)] # removing entries where no diet has been captures (@013- 3.10)
print('Percent of col removed: ', round((prior_len-len(x))/len(x),2), 'equal to ', prior_len-len(x), 'rows')
x.to_csv('overall_summaryV2.1.csv')

Percent of col removed:  0.0 equal to  0 rows


In [15]:
#summarizing the features to a daily level. Columns need to be indicated to make the choice of features easier,
#i.e. to have @column: data as the first index (columns are sorted alphabetically otherwise)
x_columns = ['date','breathing_rate_at_activity',
       'average_activity_intensity', 'classified_activityLvl',
       'classified_activity', 'c_proportion_of_sedentary_behaviors',
       'c_active_minutes_in_window', 'c_sedentary_minutes_in_window',
       'c_breathing_rate_minus_activityLvl', 'c_average_activity_intensityLvl',
       'summary_based_activityLvl', 's_proportion_of_sedentary_behaviors',
       's_active_minutes_in_window', 's_sedentary_minutes_in_window',
       's_breathing_rate_at_activityLvl', 's_average_activity_intensityLvl',
       'average_heart_rate_by_activity_intensity', 'average_heart_rate_day',
       'average_heart_rate_filtered', 'average_resting_heart_rate',
       'heart_rate_max_min', 'heart_rate_max_min_day', 'sandard_deviation_HR',
       'sandard_deviation_HR_day',
       'standard_deviation_of_the_interbeat_interval', 'fasting_glucose',
       'calories_per_day', 'unbalanced_calories',
       'calories_divided_by_mean_activity_level', 'subject_id', 'ranking']
x['date'] = [date[0:10] for date in x.Time]
sum_sum = pd.DataFrame(columns = x_columns )
for _, individual in x.groupby('subject_id'):
    for day, individual_day in individual.groupby('date'):
        day_summary = individual_day.mean()
        day_summary['date']=day
        sum_sum = sum_sum.append(day_summary, ignore_index=True)
sum_sum.to_csv('day_summary')